Production planning problem from book Operations Research - A Model-Baesed Approach 3rd Edition by H.A Eiselt and Carl-Louis Sandblom, chapter 2.2.1, page 20



In [1]:
import polars as pl

processing_times = pl.read_csv("processing_times.csv")
processing_times

machines,products,processing_times
i64,i64,i64
1,1,3
1,2,5
1,3,4
2,1,6
2,2,1
2,3,3


In [2]:
machines_availability = pl.read_csv('machines_availability.csv')
machines_availability

machines,machines_availability
i64,i64
1,540
2,480


In [3]:
products_profit = pl.read_csv("products_profit.csv")
products_profit

products,products_profit
i64,f64
1,5.0
2,3.5
3,4.5


In [4]:
import pyoframe as pf

m = pf.Model('production_planning_pyoframe', solver='highs')

m.Production = pf.Variable(products_profit[['products']], lb=0)

m.Production


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


<Variable name=Production lb=0 size=3 dimensions={'products': 3}>
[1]: Production[1]
[2]: Production[2]
[3]: Production[3]

In [5]:
machine_usage = m.Production * processing_times
machine_usage

<Expression size=6 dimensions={'products': 3, 'machines': 2} terms=6>
[1,1]: 3 Production[1]
[2,1]: 5 Production[2]
[3,1]: 4 Production[3]
[1,2]: 6 Production[1]
[2,2]: Production[2]
[3,2]: 3 Production[3]

In [6]:

m.machine_capacity = pf.sum_by('machines', machine_usage) <= machines_availability
m.machine_capacity

<Constraint name=machine_capacity sense='<=' size=2 dimensions={'machines': 2} terms=8>
[1]: 3 Production[1] +5 Production[2] +4 Production[3] <= 540
[2]: 6 Production[1] + Production[2] +3 Production[3] <= 480

In [7]:
total_profit_per_product = products_profit * m.Production
total_profit_per_product

<Expression size=3 dimensions={'products': 3} terms=3>
[1]: 5 Production[1]
[2]: 3.5 Production[2]
[3]: 4.5 Production[3]

In [8]:
m.maximize = pf.sum(total_profit_per_product)
m.objective

Hessian has dimension 4 but no nonzeros, so is ignored


<Objective size=1 dimensions={} terms=3>
objective: 5 Production[1] +3.5 Production[2] +4.5 Production[3]

In [9]:
m.optimize()
m.Production.solution

LP   has 2 rows; 4 cols; 8 nonzeros
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [4e+00, 5e+00]
  Bound  [1e+00, 1e+00]
  RHS    [0e+00, 0e+00]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 3(-1); elements 6(-2)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.1249939713e+00 Ph1: 2(7); Du: 3(4.12499) 0s
          2    -6.4000000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  6.4000000000e+02
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


products,solution
i64,f64
1,20.0
2,0.0
3,120.0


In [10]:
m.objective.value

640.0

In [11]:
m.machine_capacity.dual

machines,dual
i64,f64
1,0.8
2,0.433333


In [12]:
m.Production.attr

In [13]:
m.machine_capacity.attr

In [14]:
m.attr

In [15]:
machine_usage.sum("products").evaluate()

machines,solution
i64,f64
1,540.0
2,480.0


In [16]:
total_profit_per_product.evaluate()

products,solution
i64,f64
1,100.0
2,0.0
3,540.0
